# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 10** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez </br>
**Team**: Par de Dos  
**Members**: Diego Orozco and Aarón Ortega

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab10_ParDeDos") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 01:38:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Preparación de Datos

In [21]:
from team_ParDeDos.spark_utils import SparkUtils as SpU
# Columns: male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD

# Define schema for the DataFrame
schema = SpU.generate_schema([
        ("male", "integer"),
        ("age", "integer"),
        ("education", "integer"),
        ("currentSmoker", "integer"),
        ("cigsPerDay", "integer"),
        ("BPMeds", "integer"),
        ("prevalentStroke", "integer"),
        ("prevalentHyp", "integer"),
        ("diabetes", "integer"),
        ("totChol", "integer"),
        ("sysBP", "integer"),
        ("diaBP", "integer"),
        ("BMI", "integer"),
        ("heartRate", "integer"),
        ("glucose", "integer"),
        ("TenYearCHD", "double")
    ])

# Create DataFrame
framingham = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/framingham.csv")

clean_framingham = SpU.clean_df(framingham, schema)

### Assemble the features into a single vector column

In [22]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ["male", "age", "education", "currentSmoker", "cigsPerDay",
                  "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes",
                  "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

data_with_features = assembler.transform(clean_framingham).select("TenYearCHD", "features")

data_with_features.show(5, truncate=False)

+----------+-----------------------------------------------------------------------+
|TenYearCHD|features                                                               |
+----------+-----------------------------------------------------------------------+
|0.0       |(15,[0,1,2,9,10,11,13,14],[1.0,39.0,4.0,195.0,106.0,70.0,80.0,77.0])   |
|0.0       |(15,[1,2,9,10,11,13,14],[46.0,2.0,250.0,121.0,81.0,95.0,76.0])         |
|0.0       |[1.0,48.0,1.0,1.0,20.0,0.0,0.0,0.0,0.0,245.0,0.0,80.0,0.0,75.0,70.0]   |
|1.0       |[0.0,61.0,3.0,1.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,0.0,65.0,103.0]|
|0.0       |[0.0,46.0,3.0,1.0,23.0,0.0,0.0,0.0,0.0,285.0,130.0,84.0,0.0,85.0,85.0] |
+----------+-----------------------------------------------------------------------+
only showing top 5 rows



### Split the data into training and test sets 80% training data and 20% testing data

In [23]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [24]:
print("Original Dataset")
data_with_features.show(5, truncate=False)

# Print train dataset
print("Trained Dataset")
train_df.show(5, truncate=False)

Original Dataset
+----------+-----------------------------------------------------------------------+
|TenYearCHD|features                                                               |
+----------+-----------------------------------------------------------------------+
|0.0       |(15,[0,1,2,9,10,11,13,14],[1.0,39.0,4.0,195.0,106.0,70.0,80.0,77.0])   |
|0.0       |(15,[1,2,9,10,11,13,14],[46.0,2.0,250.0,121.0,81.0,95.0,76.0])         |
|0.0       |[1.0,48.0,1.0,1.0,20.0,0.0,0.0,0.0,0.0,245.0,0.0,80.0,0.0,75.0,70.0]   |
|1.0       |[0.0,61.0,3.0,1.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,0.0,65.0,103.0]|
|0.0       |[0.0,46.0,3.0,1.0,23.0,0.0,0.0,0.0,0.0,285.0,130.0,84.0,0.0,85.0,85.0] |
+----------+-----------------------------------------------------------------------+
only showing top 5 rows

Trained Dataset
+----------+-----------------------------------------------------------------+
|TenYearCHD|features                                                         |
+----------+-------

### Create a logistic regression model

In [25]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol="features", labelCol="TenYearCHD")


# TRAINING

In [26]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

Coefficients: [0.44442997924738903,0.06437115942304895,-0.01714871123433243,0.08886663411378709,0.01858201585886688,0.38586930633634703,0.6647169872764429,0.6242157540959542,0.6658870920934002,0.001214638645979052,0.00014111070591105773,0.004235863452192114,-0.0064485174402852926,0.0002774134475905535,0.0033566675802193364]


# PREDICTIONS

In [27]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
print("Predictions")
predictions.select("features", "prediction", "probability").show(5, truncate=False)

print("TenYearCHD")
predictions.select("features", "TenYearCHD").show(5, truncate=False)

Predictions
+----------------------------------------------------------------+----------+-----------------------------------------+
|features                                                        |prediction|probability                              |
+----------------------------------------------------------------+----------+-----------------------------------------+
|(15,[0,1,2,3,4,9,13,14],[1.0,39.0,2.0,1.0,20.0,212.0,87.0,77.0])|0.0       |[0.9360122009015007,0.06398779909849928] |
|(15,[0,1,2,3,4,9,13,14],[1.0,39.0,2.0,1.0,20.0,222.0,73.0,64.0])|0.0       |[0.9380984781466017,0.061901521853398256]|
|(15,[0,1,2,3,4,9,13,14],[1.0,43.0,1.0,1.0,20.0,206.0,86.0,89.0])|0.0       |[0.9149467215917397,0.08505327840826027] |
|(15,[0,1,2,3,4,9,13,14],[1.0,47.0,3.0,1.0,25.0,173.0,68.0,75.0])|0.0       |[0.8958145134146775,0.10418548658532245] |
|(15,[0,1,2,3,4,9,13,14],[1.0,54.0,2.0,1.0,30.0,215.0,60.0,75.0])|0.0       |[0.823770974567436,0.17622902543256402]  |
+---------------------------

In [28]:
# Show predictions and real values where prediction is 1
predictions.filter(predictions.prediction == 1).select("features", "TenYearCHD", "prediction").show(10, truncate=False)

# Show predictions and real values where prediction is 0
predictions.filter(predictions.prediction == 0).select("features", "TenYearCHD", "prediction").show(10, truncate=False)

+-----------------------------------------------------------------------+----------+----------+
|features                                                               |TenYearCHD|prediction|
+-----------------------------------------------------------------------+----------+----------+
|[1.0,52.0,3.0,1.0,35.0,0.0,0.0,1.0,1.0,281.0,133.0,93.0,0.0,115.0,80.0]|0.0       |1.0       |
|[1.0,53.0,1.0,1.0,10.0,0.0,0.0,1.0,1.0,229.0,0.0,82.0,0.0,60.0,172.0]  |0.0       |1.0       |
|[1.0,58.0,2.0,1.0,60.0,0.0,0.0,1.0,0.0,250.0,150.0,97.0,32.0,75.0,65.0]|0.0       |1.0       |
|[1.0,66.0,1.0,1.0,20.0,0.0,0.0,1.0,0.0,228.0,188.0,128.0,0.0,84.0,67.0]|0.0       |1.0       |
|(15,[0,1,2,8,9,13,14],[1.0,62.0,3.0,1.0,346.0,80.0,394.0])             |1.0       |1.0       |
|[0.0,67.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,303.0,204.0,96.0,0.0,75.0,394.0] |1.0       |1.0       |
|[1.0,62.0,1.0,1.0,23.0,0.0,0.0,1.0,0.0,286.0,164.0,88.0,0.0,85.0,126.0]|1.0       |1.0       |
|[1.0,63.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="TenYearCHD")

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

precision = evaluator.evaluate(predictions,{evaluator.metricName: "weightedPrecision"})

recall = evaluator.evaluate(predictions,{evaluator.metricName: "weightedRecall"})

f1 = evaluator.evaluate(predictions,{evaluator.metricName: "f1"})

IllegalArgumentException: label does not exist. Available: TenYearCHD, features, rawPrediction, probability, prediction

In [ ]:
sc.stop()

25/04/27 18:58:38 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-c9367a43-5ec1-4dfc-bd39-37c4c5079670. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-c9367a43-5ec1-4dfc-bd39-37c4c5079670
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.ArrayOps$.foreach$extens